In [1]:
import sys
import os
import pandas as pd
from pathlib import Path

ROOT = os.path.abspath("../")   
sys.path.append(ROOT)

from src.recommender import recommend_products


DATA_PROCESSED = Path("../data/processed")

rules = pd.read_parquet(DATA_PROCESSED / "association_rules.parquet")
rules.head()

,support,confidence,lift,antecedent_len,consequent_len,antecedents_str,consequents_str
0,0.002040,0.314286,23.323460,1,1,12 coloured party balloons,set of 9 heart shaped balloons
1,0.003091,0.409836,76.210665,1,1,12 ivory rose peg place settings,12 pink rose peg place settings
2,0.003091,0.574713,76.210665,1,1,12 pink rose peg place settings,12 ivory rose peg place settings
3,0.002843,0.377049,66.303279,1,1,12 ivory rose peg place settings,12 red rose peg place settings
4,0.002843,0.500000,66.303279,1,1,12 red rose peg place settings,12 ivory rose peg place settings


In [2]:
# Define a helper function that converts strings to Python sets

def str_to_set(s):
    """
    Convert 'a, b, c' into {'a', 'b', 'c'}
    """
    return set(x.strip() for x in s.split(","))

In [3]:
rules["antecedents_set"] = rules["antecedents_str"].apply(str_to_set)
rules["consequents_set"] = rules["consequents_str"].apply(str_to_set)

rules[["antecedents_str", "antecedents_set", "consequents_set"]].head()

,antecedents_str,antecedents_set,consequents_set
0,12 coloured party balloons,{12 coloured party balloons},{set of 9 heart shaped balloons}
1,12 ivory rose peg place settings,{12 ivory rose peg place settings},{12 pink rose peg place settings}
2,12 pink rose peg place settings,{12 pink rose peg place settings},{12 ivory rose peg place settings}
3,12 ivory rose peg place settings,{12 ivory rose peg place settings},{12 red rose peg place settings}
4,12 red rose peg place settings,{12 red rose peg place settings},{12 ivory rose peg place settings}


In [4]:
# Test 1
basket1 = [
    "12 pencil small tube woodland",
    "3 hanger"
]

recommend_products(basket1, rules, top_n=3)

['12 pencils small tube skull']

In [5]:
# Test 2: Multi-item basket
recommend_products(
    ["12 red rose peg place settings", "3 hearts hanging decoration rustic", "12 pencils small tube skull"],
    rules,
    top_n=3
)

['12 ivory rose peg place settings',
 '12 pink rose peg place settings',
 'white hanging heart t-light holder']

In [6]:
# Test 3: Single basket
recommend_products(["12 coloured party balloons"], rules, 5)

['set of 9 heart shaped balloons']

In [7]:
# Test 4
recommend_products(["this product does not exist"], rules)

[]

In [8]:
sample = {
    "basket_example": basket1,
    "recommendations": recommend_products(basket1, rules, top_n=3)
}

pd.Series(sample).to_json(DATA_PROCESSED / "sample_recommendations.json")